# Import

In [122]:
!pip install pandas

In [123]:
import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "iteration_utilities"])

0

In [124]:
import pandas as pd

from iteration_utilities import duplicates

On appelle le generateur de preference 

In [125]:
subprocess.call("generator.py", shell=True)

0

# Variables

Entree : 

    Deux ensembles d'elèves et d'etablissement avec leur preference;

    Une famille L de relations de preferences ;

In [126]:
#prefEtudiant = pd.read_csv("fichierEleves.csv", header=None, sep = ",").values.tolist() 

#prefEtablissement = pd.read_csv("fichierEtablissements.csv", header=None, sep = ",").values.tolist()

In [ ]:
prefEtudiant = pd.read_csv("fichierEtablissements.csv", header=None, sep = ",").values.tolist() 

prefEtablissement = pd.read_csv("fichierEleves.csv", header=None, sep = ",").values.tolist()

# Algorithme


Etape 0 : chaque elève soumet une liste ordonnee de vœux  
Etape 1 : on ne considère que les vœux de rang 1  
 * chaque ecole considère les elèves qui l’ont classee en 1er vœu  
 * chaque ecole accepte temporairement les mieux classes dans la limite des places disponibles et rejette les autres.  

. . .  

Etape k : les elèves rejetes à l’etape precedente candidatent
sur leur vœu suivant  
 * chaque ecole considère conjointement les elèves precedemment
admis et les elèves lui faisant une offre à cette etape  
 * les mieux classes sont temporairement acceptes et les autres
sont rejetes  
L’algorithme se termine au bout d’un nombre fini d’iterations
lorsque plus aucun elève n’est rejete.

In [127]:
file = open("resultat.txt", "w") 

In [128]:
def attribution(n, prefEtudiant, prefEtablissement):
    print("Tous les etudiants sont libres au debut : ", file=file)
    etudiantLibre = list(range(n))
    print("Liste des etudiants libres : {}".format(etudiantLibre), file=file)
    # Aucun etudiant n'a d'affectation
    affectationEtudiant = [None] * n     
    print("affectation etudiant {}".format(affectationEtudiant), file=file)                 
    # Aucun etablissement n'a d'etudiant
    affectationEtablissement = [None] * n  
    print("affectation etablissement {}".format(affectationEtablissement), file=file)                    
    # Aucun etudiant n'a fait de proposition on est donc sur le choix 1 (0) de chaque etudiant
    suivant = [0] * n  
    print("suivant : {}".format(suivant), file=file)                     
    etape = 0
    # Tant que tous les etudiants n'ont pas d'affectation:
    while etudiantLibre:
        etape = etape +1
        print("  ", file=file)
        print("-------------------------------------------------------------------------------------------", file=file)
        print("  ", file=file)
        print("Etape : {}".format(etape), file=file)
        # On prend le premier des etudiants lilbres
        etudiant = etudiantLibre[0] 
        print("On prend l'etudiant : {}".format(etudiant), file=file)                     
        # On prend ses preferences
        prefEtudiantLibre = prefEtudiant[etudiant]  
        print("Preferences de l'etudiant {}".format(etudiant)," : {}".format(prefEtudiantLibre) , file=file)
        # On prend le choix n de l'etudiant
        etablissement = prefEtudiantLibre[suivant[etudiant]] 
        print("Le choix numero : {}".format(suivant[etudiant])," de l'etudiant : {}".format(etudiant) ," est  : {}".format(etablissement), file=file)
        # On prend le classement de l'elève
        prefEtablissementEleve = prefEtablissement[etablissement]
        print("Classement de l'elève : {}".format(prefEtablissementEleve), file=file)
        # On prend l'etudiant actuellement affecte à cet etablissement
        etudiantActuel = affectationEtablissement[etablissement]
       
        
        if etudiantActuel == None:
          #Si il n'y a pas d'etudiant dejà affecte à cet etablissement la demande est accepte
          affectationEtablissement[etablissement] = etudiant
          affectationEtudiant[etudiant] = etablissement
          print("Aucun etudiant n'est dejà affecte a cet etablissement.", file=file)
          print("{}".format(etudiant)," affecte à l'etablissement : {}".format(etablissement), file=file)
          print("Affectation des etablissements : {}".format(affectationEtablissement), file=file)
          print("Affectation des etudiants : {}".format(affectationEtudiant), file=file)
          #Son choix suivant passe à la femme suivante dans sa liste
          suivant[etudiant] = suivant[etudiant] + 1
          #On l'enlève de la liste des etudiants libres
          etudiantLibre.pop(0)
        else:
          #Si il y a dejà un etudiant affecte à cet etablissement
          print("Preferences de l'etablissement : {}".format(prefEtablissementEleve), file=file)
          print("L'etudiant : {}".format(etudiantActuel)," est deja affecte à l'etablissement {}".format(etablissement) , file=file)

          # On regarde les preferences de l'etablissement pour les 2 elèves
          indexEtudiantActuel = prefEtablissementEleve.index(etudiantActuel)
          
          indexEtudiant2 = prefEtablissementEleve.index(etudiant)
          
          if indexEtudiantActuel > indexEtudiant2:
             #L'etablissement prefere le nouvel etudiant
             print("{}".format(etudiant)," est prefere à {}".format(etudiantActuel),  file=file)
             affectationEtablissement[etablissement] = etudiant
             print("{}".format(etudiant)," affecte à l'etablissement : {}".format(etablissement), file=file)
             affectationEtudiant[etudiant] = etablissement
             suivant[etudiant] = suivant[etudiant] + 1
             #On l'enlève de la liste des etudiants libres
             etudiantLibre.pop(0)
             #L'autre etudiant n'a plus d'etablissement
             etudiantLibre.insert(0,etudiantActuel)
          else:
            print("{}".format(etudiantActuel)," est prefere à {}".format(etudiant), file=file)
            print("Son choix {}".format(suivant[etudiant])," est refuse {}", file=file)
            suivant[etudiant] = suivant[etudiant] + 1
             
           
    
  
    return affectationEtudiant

Sortie :

    Liste des affectations par couple

In [129]:
res = attribution(len(prefEtudiant), prefEtudiant, prefEtablissement)
print("  ", file=file)
print("----------------------------------RESULTATS-----------------------------------------", file=file)
print("  ", file=file)
print("{}".format(res), file= file)
print(res)
print("Le détail du résultat est dans resultat.txt")


[64, 932, 302, 552, 254, 481, 316, 456, 627, 95, 167, 658, 532, 555, 745, 791, 280, 891, 969, 203, 904, 4, 917, 786, 462, 186, 841, 933, 63, 138, 149, 693, 124, 701, 668, 907, 820, 333, 974, 538, 990, 684, 955, 757, 685, 790, 956, 855, 623, 557, 612, 524, 54, 251, 723, 74, 200, 601, 9, 864, 630, 129, 998, 2, 217, 440, 131, 154, 269, 908, 657, 469, 906, 430, 389, 198, 507, 359, 168, 533, 42, 709, 155, 59, 253, 772, 127, 459, 741, 60, 215, 325, 340, 315, 205, 286, 457, 697, 231, 980, 988, 846, 472, 356, 512, 99, 393, 671, 599, 303, 400, 817, 174, 139, 240, 877, 762, 503, 375, 148, 609, 579, 282, 924, 529, 93, 564, 111, 578, 386, 735, 22, 34, 202, 553, 766, 485, 427, 879, 852, 554, 611, 412, 794, 101, 351, 953, 950, 582, 561, 628, 134, 978, 497, 776, 952, 886, 58, 960, 721, 152, 97, 490, 78, 914, 435, 548, 55, 897, 328, 987, 109, 895, 758, 305, 265, 707, 128, 388, 866, 661, 748, 165, 56, 169, 832, 784, 839, 345, 637, 32, 896, 354, 652, 483, 662, 68, 916, 732, 966, 787, 870, 276, 498, 706,

On teste s'il n'y a pas de doublons

In [130]:
print("{}".format(list(duplicates(res))))
# Si la fonction renvoie [] s'il n'y a pas de doublons

[]


In [131]:
file.close()

In [132]:
# retourne la somme des priorités (0, 1, 2) recues dans cette attribution
def etudiantSatisfaction(n, attribution, prefEtudiant, prefEtablissement, fonction):
    result = 0
    for etudiant,etablissement in enumerate(attribution):
        result = result + prefEtudiant[etudiant].index(etablissement) # ajoute 0, 1 ou 2
        # print(prefEtudiant[etudiant].indeetablissement))
    return result 

def etablissmentSatisfaction(n, attribution, prefEtudiant, prefEtablissement, fonction):
    result = 0
    for etudiant,etablissement in enumerate(attribution):
        result = result + prefEtablissement[etablissement].index(etudiant) # ajoute 0, 1 ou 2
        # print(prefEtudiant[etudiant].indeetablissement))
    return result 

def fonctionLineaire(x):
    return x

print(etudiantSatisfaction(len(prefEtudiant), res, prefEtudiant, prefEtablissement, fonctionLineaire))
etablissmentSatisfaction(len(prefEtudiant), res, prefEtudiant, prefEtablissement, fonctionLineaire)

7231


126200